In [1]:
gameFolder = "DOA++-MAME"

In [2]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../games',gameFolder))   

tensorBoardFolder = "./ppo2_TB_CustCnnLnLstm/"
modelFolder = "ppo2_Model_CustCnnLnLstm/"

In [3]:
from makeDiambraEnv import *

import tensorflow as tf

%load_ext tensorboard

from stable_baselines.common.policies import RecurrentActorCriticPolicy
from stable_baselines.common.policies import *
from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.callbacks import BaseCallback

In [4]:
class CustomCnnLstmPolicy(RecurrentActorCriticPolicy):
    """
    Policy object that implements actor critic, using LSTMs.

    :param sess: (TensorFlow session) The current TensorFlow session
    :param ob_space: (Gym Space) The observation space of the environment
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batch to run (n_envs * n_steps)
    :param n_lstm: (int) The number of LSTM cells (for recurrent policies)
    :param reuse: (bool) If the policy is reusable or not
    :param layers: ([int]) The size of the Neural network before the LSTM layer  (if None, default to [64, 64])
    :param net_arch: (list) Specification of the actor-critic policy network architecture. Notation similar to the
        format described in mlp_extractor but with additional support for a 'lstm' entry in the shared network part.
    :param act_fun: (tf.func) the activation function to use in the neural network.
    :param cnn_extractor: (function (TensorFlow Tensor, ``**kwargs``): (TensorFlow Tensor)) the CNN feature extraction
    :param layer_norm: (bool) Whether or not to use layer normalizing LSTMs
    :param feature_extraction: (str) The feature extraction type ("cnn" or "mlp")
    :param kwargs: (dict) Extra keyword arguments for the nature CNN feature extraction
    """

    recurrent = True

    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, n_lstm=256, reuse=False, 
                 layers=None, net_arch=None, act_fun=tf.tanh, cnn_extractor=nature_cnn, 
                 layer_norm=True, feature_extraction="cnn", **kwargs):
        
        # state_shape = [n_lstm * 2] dim because of the cell and hidden states of the LSTM
        super(CustomCnnLstmPolicy, self).__init__(sess, ob_space, ac_space, n_env, n_steps, n_batch,
                                         state_shape=(2 * n_lstm, ), reuse=reuse,
                                         scale=(feature_extraction == "cnn"))

      
        self._kwargs_check(feature_extraction, kwargs)
       
        frames = self.processed_obs[:,:,:,0:4]
        additional_input = self.processed_obs[:,:,:,4]
        additional_input = tf.layers.flatten(additional_input)
        additional_input = additional_input[:,1:149]
        
        if layers is None:
            layers = [64, 64]
        else:
            warnings.warn("The layers parameter is deprecated. Use the net_arch parameter instead.")

        with tf.variable_scope("model", reuse=reuse):
            
            # Frames (CNN)
            extracted_features_frames = cnn_extractor(frames, **kwargs)
            
            # Additional (Additional Info)
            for i, layer_size in enumerate(layers):
                extracted_features_addinfo = act_fun(linear(additional_input, 'pi_fc' + str(i), 
                                                            n_hidden=layer_size, init_scale=np.sqrt(2)))
            
            extracted_features = tf.concat([extracted_features_frames, extracted_features_addinfo], 1)
            
            #print("Extracted feat frames")
            #print(extracted_features_frames)
            #print("Extracted feat additional info")
            #print(extracted_features_addinfo)
            #print("Extracted feat")
            #print(extracted_features)
            input_sequence = batch_to_seq(extracted_features, self.n_env, n_steps)
            #print("input_sequence")
            #print(input_sequence)
            #print("dones_ph")
            #print(self.dones_ph)
            masks = batch_to_seq(self.dones_ph, self.n_env, n_steps)
            #print("masks")
            #print(masks)
            rnn_output, self.snew = lstm(input_sequence, masks, self.states_ph, 'lstm1', n_hidden=n_lstm,
                                         layer_norm=layer_norm)
            #print("rnn_output")
            #print(rnn_output)
            #input("Pause")
            rnn_output = seq_to_batch(rnn_output)
            value_fn = linear(rnn_output, 'vf', 1)

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(rnn_output, rnn_output)

        self._value_fn = value_fn
        
        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            return self.sess.run([self.deterministic_action, self.value_flat, self.snew, self.neglogp],
                                 {self.obs_ph: obs, self.states_ph: state, self.dones_ph: mask})
        else:
            return self.sess.run([self.action, self.value_flat, self.snew, self.neglogp],
                                 {self.obs_ph: obs, self.states_ph: state, self.dones_ph: mask})

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs, self.states_ph: state, self.dones_ph: mask})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs, self.states_ph: state, self.dones_ph: mask})


In [5]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 3
#diambraKwargs["render"] =True
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] = "Kasumi"

wrapperKwargs = {}
wrapperKwargs["frame_stack"] = 4
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["scale"] = True
wrapperKwargs["hw_obs_resize"] = [256, 256]

#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBuf")
#keyToAdd.append("player")
keyToAdd.append("healthP1")
keyToAdd.append("healthP2")
keyToAdd.append("positionP1")
keyToAdd.append("positionP2")
#keyToAdd.append("winsP1")
#keyToAdd.append("winsP2")

numEnv=1

env = make_diambra_env(diambraMame, env_prefix="Train", num_env=numEnv, seed=timeDepSeed, 
                       continue_game = -2, diambra_kwargs = diambraKwargs, 
                       wrapper_kwargs = wrapperKwargs, key_to_add = keyToAdd)

# OR 
#env = make_diambra_env(diambraMame, num_env=2, seed=0, diambra_kwargs = diambraKwargs)
# Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)


Env_id =  Train0
Continue rule =  False
Player = P1 , Character = Kasumi
Noop action N =  11


In [ ]:
print("Obs_space = ", env.observation_space)
print("Obs_space type = ", env.observation_space.dtype)
print("Obs_space high = ", env.observation_space.high)
print("Obs_space low = ", env.observation_space.low)

In [ ]:
print("Act_space = ", env.action_space)
print("Act_space type = ", env.action_space.dtype)
print("Act_space n = ", env.action_space.n)

In [6]:
policyKwargs={}
policyKwargs["layers"] = [128, 256]

# Initialize the model, 1 env
#model = PPO2(CustomCnnLstmPolicy, env, nminibatches=1, verbose=1,
#             tensorboard_log=tensorBoardFolder, policy_kwargs=policyKwargs, gamma = 0.9)

#OR

# Load the trained agent, 1 env
model = PPO2.load(modelFolder+"4_3Msteps_action+_add", env=env, tensorboard_log=tensorBoardFolder, 
                  policy_kwargs=policyKwargs, gamma = 0.94)





Instructions for updating:
Use keras.layers.flatten instead.


/home/alexpalms/anaconda3/envs/diambra/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: The layers parameter is deprecated. Use the net_arch parameter instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
# Train the agent
time_steps = 10000000
model.learn(total_timesteps=time_steps)


Setting difficulty = 3
Starting game
Selecting Kasumi
Waiting for fight to start
---------------------------------------
| approxkl           | 2.5212782e-06  |
| clipfrac           | 0.0            |
| explained_variance | 0.0804         |
| fps                | 2              |
| n_updates          | 1              |
| policy_entropy     | 2.3138862      |
| policy_loss        | -0.00044920295 |
| serial_timesteps   | 128            |
| time_elapsed       | 3.65e-05       |
| total_timesteps    | 128            |
| value_loss         | 0.0015072183   |
---------------------------------------
--------------------------------------
| approxkl           | 1.5704218e-06 |
| clipfrac           | 0.0           |
| explained_variance | -0.0114       |
| fps                | 23            |
| n_updates          | 2             |
| policy_entropy     | 2.358452      |
| policy_loss        | -0.0002909582 |
| serial_timesteps   | 256           |
| time_elapsed       | 45.3          |
| total_

Episode done
Restarting game
Selecting Kasumi
Waiting for fight to start
---------------------------------------
| approxkl           | 1.4725254e-06  |
| clipfrac           | 0.0            |
| ep_len_mean        | 2.12e+03       |
| ep_reward_mean     | 1.21           |
| explained_variance | 0.202          |
| fps                | 8              |
| n_updates          | 17             |
| policy_entropy     | 2.3247085      |
| policy_loss        | -0.00018822774 |
| serial_timesteps   | 2176           |
| time_elapsed       | 159            |
| total_timesteps    | 2176           |
| value_loss         | 0.013077898    |
---------------------------------------
---------------------------------------
| approxkl           | 1.9347656e-06  |
| clipfrac           | 0.0            |
| ep_len_mean        | 2.12e+03       |
| ep_reward_mean     | 1.21           |
| explained_variance | 0.0171         |
| fps                | 23             |
| n_updates          | 18             |
| polic

--------------------------------------
| approxkl           | 3.4124795e-07 |
| clipfrac           | 0.0           |
| ep_len_mean        | 2.12e+03      |
| ep_reward_mean     | 1.21          |
| explained_variance | -5.95e-05     |
| fps                | 21            |
| n_updates          | 31            |
| policy_entropy     | 2.3631635     |
| policy_loss        | -0.0001604706 |
| serial_timesteps   | 3968          |
| time_elapsed       | 283           |
| total_timesteps    | 3968          |
| value_loss         | 0.007088354   |
--------------------------------------
--------------------------------------
| approxkl           | 4.9124344e-07 |
| clipfrac           | 0.0           |
| ep_len_mean        | 2.12e+03      |
| ep_reward_mean     | 1.21          |
| explained_variance | 0.00174       |
| fps                | 23            |
| n_updates          | 32            |
| policy_entropy     | 2.362805      |
| policy_loss        | -8.691102e-05 |
| serial_timesteps   | 40

--------------------------------------
| approxkl           | 5.4171375e-07 |
| clipfrac           | 0.0           |
| ep_len_mean        | 1.74e+03      |
| ep_reward_mean     | 1.78          |
| explained_variance | -0.00314      |
| fps                | 24            |
| n_updates          | 45            |
| policy_entropy     | 2.3594697     |
| policy_loss        | -8.163694e-05 |
| serial_timesteps   | 5760          |
| time_elapsed       | 400           |
| total_timesteps    | 5760          |
| value_loss         | 0.0015930282  |
--------------------------------------
---------------------------------------
| approxkl           | 4.2738517e-07  |
| clipfrac           | 0.0            |
| ep_len_mean        | 1.74e+03       |
| ep_reward_mean     | 1.78           |
| explained_variance | 0.0112         |
| fps                | 23             |
| n_updates          | 46             |
| policy_entropy     | 2.36108        |
| policy_loss        | -4.2196363e-05 |
| serial_timest

--------------------------------------
| approxkl           | 3.3337687e-07 |
| clipfrac           | 0.0           |
| ep_len_mean        | 1.74e+03      |
| ep_reward_mean     | 1.78          |
| explained_variance | -0.00161      |
| fps                | 23            |
| n_updates          | 59            |
| policy_entropy     | 2.3672385     |
| policy_loss        | -8.16565e-05  |
| serial_timesteps   | 7552          |
| time_elapsed       | 502           |
| total_timesteps    | 7552          |
| value_loss         | 0.008260985   |
--------------------------------------
--------------------------------------
| approxkl           | 2.8108443e-07 |
| clipfrac           | 0.0           |
| ep_len_mean        | 1.74e+03      |
| ep_reward_mean     | 1.78          |
| explained_variance | 0.00285       |
| fps                | 23            |
| n_updates          | 60            |
| policy_entropy     | 2.3666558     |
| policy_loss        | -3.874302e-06 |
| serial_timesteps   | 76

...

--------------------------------------
| approxkl           | 1.3944173e-06 |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.27e+03      |
| ep_reward_mean     | 4.08          |
| explained_variance | -0.000424     |
| fps                | 21            |
| n_updates          | 1935          |
| policy_entropy     | 2.206231      |
| policy_loss        | 3.5137404e-05 |
| serial_timesteps   | 247680        |
| time_elapsed       | 1.54e+04      |
| total_timesteps    | 247680        |
| value_loss         | 0.010391757   |
--------------------------------------
Round done
--------------------------------------
| approxkl           | 3.613781e-07  |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.27e+03      |
| ep_reward_mean     | 4.08          |
| explained_variance | 0.000182      |
| fps                | 10            |
| n_updates          | 1936          |
| policy_entropy     | 2.2075353     |
| policy_loss        | 7.3611736e-05 |
| serial_times

Round done
--------------------------------------
| approxkl           | 1.7125823e-06 |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.27e+03      |
| ep_reward_mean     | 4.1           |
| explained_variance | 0.00181       |
| fps                | 10            |
| n_updates          | 1949          |
| policy_entropy     | 2.203922      |
| policy_loss        | -8.925982e-05 |
| serial_timesteps   | 249472        |
| time_elapsed       | 1.55e+04      |
| total_timesteps    | 249472        |
| value_loss         | 0.005796832   |
--------------------------------------
--------------------------------------
| approxkl           | 1.15398e-06   |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.27e+03      |
| ep_reward_mean     | 4.1           |
| explained_variance | -0.00448      |
| fps                | 21            |
| n_updates          | 1950          |
| policy_entropy     | 2.207725      |
| policy_loss        | -7.987581e-05 |
| serial_times

Stage done
---------------------------------------
| approxkl           | 1.983541e-06   |
| clipfrac           | 0.0            |
| ep_len_mean        | 3.27e+03       |
| ep_reward_mean     | 4.1            |
| explained_variance | 0.000552       |
| fps                | 9              |
| n_updates          | 1963           |
| policy_entropy     | 2.2397065      |
| policy_loss        | -0.00019458309 |
| serial_timesteps   | 251264         |
| time_elapsed       | 1.57e+04       |
| total_timesteps    | 251264         |
| value_loss         | 0.007431876    |
---------------------------------------
--------------------------------------
| approxkl           | 1.2636594e-06 |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.27e+03      |
| ep_reward_mean     | 4.1           |
| explained_variance | 0.000875      |
| fps                | 23            |
| n_updates          | 1964          |
| policy_entropy     | 2.2372293     |
| policy_loss        | 1.6892329e-05 |

Round done
--------------------------------------
| approxkl           | 1.3666404e-06 |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.28e+03      |
| ep_reward_mean     | 4.1           |
| explained_variance | -0.00255      |
| fps                | 11            |
| n_updates          | 1977          |
| policy_entropy     | 2.1998544     |
| policy_loss        | -7.075677e-05 |
| serial_timesteps   | 253056        |
| time_elapsed       | 1.58e+04      |
| total_timesteps    | 253056        |
| value_loss         | 0.010657747   |
--------------------------------------
---------------------------------------
| approxkl           | 7.8705045e-07  |
| clipfrac           | 0.0            |
| ep_len_mean        | 3.28e+03       |
| ep_reward_mean     | 4.1            |
| explained_variance | -0.00435       |
| fps                | 22             |
| n_updates          | 1978           |
| policy_entropy     | 2.2067444      |
| policy_loss        | -5.4568052e-05 |
| se

--------------------------------------
| approxkl           | 2.0225824e-07 |
| clipfrac           | 0.0           |
| ep_len_mean        | 3.28e+03      |
| ep_reward_mean     | 4.1           |
| explained_variance | 0.00228       |
| fps                | 24            |
| n_updates          | 1991          |
| policy_entropy     | 2.193215      |
| policy_loss        | -3.297627e-05 |
| serial_timesteps   | 254848        |
| time_elapsed       | 1.59e+04      |
| total_timesteps    | 254848        |
| value_loss         | 0.0061155716  |
--------------------------------------
Episode done
Restarting game
Selecting Kasumi
Waiting for fight to start


In [ ]:
# Save the agent
model.save(modelFolder+"4_3Msteps_action+_add")

In [ ]:
%tensorboard --logdir ./tensorBoardFolder/

In [ ]:
observation = env.reset()
states = None

while True:

    action, states = model.predict(observation, states, deterministic=False)
    action_prob = model.action_probability(observation, states)
    print("Action probabilities = ", action_prob)
    print("Max action = ", np.argmax(action_prob))
    print("Action = ", action)
    input("Pausa")
    
    observation, reward, done, info = env.step(action)
    if done:
        observation = env.reset()
        states = None
        
env.close()

In [ ]:
observation = env.reset()
states = None

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    action, states = model.predict(observation, states, deterministic=False)
    action_prob = model.action_probability(observation, states)
    print("Action probabilities = ", action_prob)
    print("Max action = ", np.argmax(action_prob))
    print("Action = ", action)
    
    observation, reward, done, info = env.step(action)
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

        observation = env.reset()
        states = None

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))   
    
env.close()